# Books data scraping
## https://www.goodreads.com/list

In [1]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import regex as re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
browser = Browser('chrome')

In [3]:
# Visit the website for the genres list
url = "https://www.goodreads.com/genres/list?page=1"
browser.visit(url)

In [4]:
# Extract all genres on main list page
genre_list=[]
genre_links_list=[]
is_next_page=True
while (is_next_page):
    html = browser.html
    html_objct = soup(html, 'html.parser')
    genre_items=html_objct.select("div.leftContainer div.shelfStat a.actionLinkLite")
    for i in genre_items:
        genre_list.append(i.text)
        temp_link="https://www.goodreads.com/search?q="+i.text
        genre_links_list.append(temp_link)
    time.sleep(2)    
    try:
        browser.links.find_by_partial_text('next »').click()
    except: 
        try:
            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_experimental_option('useAutomationExtension', False)
            browser = webdriver.Chrome('./chromedriver', options=chrome_options)
            browser.get((url))
            button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[img[alt="Dismiss"]]')))
            button.click()
        except: is_next_page = False      

genre_links_list

['https://www.goodreads.com/search?q=10th-century',
 'https://www.goodreads.com/search?q=11th-century',
 'https://www.goodreads.com/search?q=12th-century',
 'https://www.goodreads.com/search?q=13th-century',
 'https://www.goodreads.com/search?q=14th-century',
 'https://www.goodreads.com/search?q=15th-century',
 'https://www.goodreads.com/search?q=16th-century',
 'https://www.goodreads.com/search?q=17th-century',
 'https://www.goodreads.com/search?q=1864-shenandoah-campaign',
 'https://www.goodreads.com/search?q=18th-century',
 'https://www.goodreads.com/search?q=1917',
 'https://www.goodreads.com/search?q=19th-century',
 'https://www.goodreads.com/search?q=1st-grade',
 'https://www.goodreads.com/search?q=20th-century',
 'https://www.goodreads.com/search?q=21st-century',
 'https://www.goodreads.com/search?q=2nd-grade',
 'https://www.goodreads.com/search?q=40k',
 'https://www.goodreads.com/search?q=ableism',
 'https://www.goodreads.com/search?q=abuse',
 'https://www.goodreads.com/search?

In [5]:
books=[]
is_next_page=True
# for i in range(len(genre_links_list)):
# try to scrape few genres only
for i in range(10):
    book_genre=genre_list[i]
    browser.visit(genre_links_list[i])
    is_next_page=True
    page_counter=0
    while (is_next_page):
        html = browser.html
        html_objct = soup(html, 'html.parser')
        book_row=html_objct.select('table.tableList tr')
        for r in book_row:
            book={}
            book['name']=r.select_one('a.bookTitle>span[itemprop="name"]').text
            book['details_link']='https://www.goodreads.com'+r.select_one('a.bookTitle[itemprop="url"]')['href']
            authors=r.select('span[itemprop="author"] div.authorName__container')
            authors_list=[]
            for j in authors:
                authors_list.append(j.select_one('a.authorName>span[itemprop="name"]').text)
            book['authors']=authors_list
            try:
                all_rating=r.select_one('span.minirating span.stars').next_sibling.strip()
                book['avg_rating'] = float(re.findall(r'(\d+(\.\d+){0,1})(?= avg rating —)', all_rating)[0][0])
                book['count_ratings'] = int(re.findall(r'(\d+)(?= rating(s)?)', all_rating)[0][0])
            except: book['count_ratings']=0
            editions_text = r.find('a', class_='greyText', rel='nofollow').get_text()
            editions_part = re.search(r'(\d+) editions', editions_text)
            if editions_part:
                book['editions_count'] = int(editions_part.group(1))
            else:
                book['editions_count'] = 1  
            book['genre']=book_genre
            books.append(book)
        if page_counter>2:
            is_next_page=False
        time.sleep(1) 

        try:
            browser.links.find_by_partial_text('next »').click()
            page_counter+=1
        except:
            is_next_page=False
books[0:3]

[{'name': "The Animals' Lawsuit Against Humanity: An Illustrated 10th Century Iraqi Ecological Fable",
  'details_link': 'https://www.goodreads.com/book/show/183006.The_Animals_Lawsuit_Against_Humanity?from_search=true&from_srp=true&qid=NKupltxWtR&rank=1',
  'authors': ['Ikhwan al-Safa', 'Rabbi Dan Bridge', 'Rabbi Kalonymus'],
  'avg_rating': 4.25,
  'count_ratings': 171,
  'editions_count': 13,
  'genre': '10th-century'},
 {'name': "The Animals' Lawsuit Against Humanity: An Illustrated 10th Century Iraqi Ecological Fable",
  'details_link': 'https://www.goodreads.com/book/show/19544232-the-animals-lawsuit-against-humanity?from_search=true&from_srp=true&qid=NKupltxWtR&rank=2',
  'authors': ['Ikhwan al-Safa', 'Umm Kulthum', 'Anson Laytner'],
  'avg_rating': 4.23,
  'count_ratings': 26,
  'editions_count': 1,
  'genre': '10th-century'},
 {'name': 'The Formation of the Sunni Schools of Law, 9th-10th Centuries C.E. (Studies in Islamic Law and Society, V. 4)',
  'details_link': 'https://www

In [6]:
len(books)

400

In [7]:
for i in books[:100]:
    refresh_attempt = 0  
    page_is_unavailable=True

    while (page_is_unavailable and (refresh_attempt < 2)):  
        browser.visit(i['details_link'])
        html = browser.html
        html_objct = soup(html, 'html.parser')

        unavailable = html_objct.select_one('div.mainContent h1')
        if unavailable and unavailable.text.strip() == 'page unavailable':
            time.sleep(1500)  
            refresh_attempt += 1 
            continue
        else: page_is_unavailable=False
    if not page_is_unavailable:
        try:
            i['reviews']=html_objct.select_one('span[data-testid="reviewsCount"]').text.strip().split()[0]
        except Exception as error_content:
            print(f"Here {i['details_link']} we have '{error_content}' within reviews")
            i['reviews']=0

        try: 
            price_content = html_objct.select_one('button.Button--buy span.Button__labelItem').text.strip()
            if '$' in price_content:
                content_price_list=price_content.split()
                i['price'] = content_price_list[-1][1:]
                i['price_type'] = content_price_list[:-2]
            else:
                i['price']=None
                i['price_type'] = None
        except Exception as error_content:
            print(f"Here {i['details_link']} we have '{error_content}' within price_kindle") 
            i['price_kindle']=None

        try:
            i['pages_count']=html_objct.select_one('div.FeaturedDetails p[data-testid="pagesFormat"]').text.strip().split()[0]
        except Exception as error_content:
            print(f"Here {i['details_link']} we have '{error_content}' within pages_count") 
            i['pages_count']=None

        try:
            i['year_first_published']=html_objct.select_one('div.FeaturedDetails p[data-testid="publicationInfo"]').text.strip().split()[-1]
        except Exception as error_content:
            print(f"Here {i['details_link']} we have '{error_content}' within year_first_published") 
            i['year_first_published']=None
        time.sleep(3)
    else: break
books 


Here https://www.goodreads.com/book/show/28165599-sogdians-in-china-archaeological-and-art-historical-analyses-of-tombs-a?from_search=true&from_srp=true&qid=NKupltxWtR&rank=25 we have ''NoneType' object has no attribute 'text'' within pages_count
Here https://www.goodreads.com/book/show/28165599-sogdians-in-china-archaeological-and-art-historical-analyses-of-tombs-a?from_search=true&from_srp=true&qid=NKupltxWtR&rank=25 we have ''NoneType' object has no attribute 'text'' within year_first_published
Here https://www.goodreads.com/book/show/6641105-nareg---a-10th-century-monk-s-conversations-with-god?from_search=true&from_srp=true&qid=NKupltxWtR&rank=32 we have ''NoneType' object has no attribute 'text'' within pages_count
Here https://www.goodreads.com/book/show/6641105-nareg---a-10th-century-monk-s-conversations-with-god?from_search=true&from_srp=true&qid=NKupltxWtR&rank=32 we have ''NoneType' object has no attribute 'text'' within year_first_published
Here https://www.goodreads.com/boo

[{'name': "The Animals' Lawsuit Against Humanity: An Illustrated 10th Century Iraqi Ecological Fable",
  'details_link': 'https://www.goodreads.com/book/show/183006.The_Animals_Lawsuit_Against_Humanity?from_search=true&from_srp=true&qid=NKupltxWtR&rank=1',
  'authors': ['Ikhwan al-Safa', 'Rabbi Dan Bridge', 'Rabbi Kalonymus'],
  'avg_rating': 4.25,
  'count_ratings': 171,
  'editions_count': 13,
  'genre': '10th-century',
  'reviews': '24',
  'price': '0.00',
  'price_type': ['Kindle'],
  'pages_count': '170',
  'year_first_published': '950'},
 {'name': "The Animals' Lawsuit Against Humanity: An Illustrated 10th Century Iraqi Ecological Fable",
  'details_link': 'https://www.goodreads.com/book/show/19544232-the-animals-lawsuit-against-humanity?from_search=true&from_srp=true&qid=NKupltxWtR&rank=2',
  'authors': ['Ikhwan al-Safa', 'Umm Kulthum', 'Anson Laytner'],
  'avg_rating': 4.23,
  'count_ratings': 26,
  'editions_count': 1,
  'genre': '10th-century',
  'reviews': '3',
  'price': No

In [8]:
genre_list

['10th-century',
 '11th-century',
 '12th-century',
 '13th-century',
 '14th-century',
 '15th-century',
 '16th-century',
 '17th-century',
 '1864-shenandoah-campaign',
 '18th-century',
 '1917',
 '19th-century',
 '1st-grade',
 '20th-century',
 '21st-century',
 '2nd-grade',
 '40k',
 'ableism',
 'abuse',
 'academia',
 'academic',
 'academics',
 'accounting',
 'accra',
 'action',
 'activism',
 'adaptations',
 'addis-ababa',
 'addition',
 'adhd',
 'adolescence',
 'adoption',
 'adult',
 'adult-colouring-books',
 'adult-fiction',
 'adventure',
 'adventurers',
 'aeroplanes',
 'africa',
 'african-american',
 'african-american-literature',
 'african-american-romance',
 'african-literature',
 'agender',
 'agriculture',
 'ahistory',
 'aircraft',
 'airliners',
 'airships',
 'albanian-literature',
 'alchemy',
 'alcohol',
 'alexandria',
 'algebra',
 'algeria',
 'algiers',
 'algorithms',
 'aliens',
 'alternate-history',
 'alternate-universe',
 'alternative-medicine',
 'amateur-sleuth',
 'amazon',
 'ambul

### Store the Data and Prepare for Analysis

Assemble the scraped data into a Pandas DataFrame. 
Examine the data types that are currently associated with each column. 

In [9]:
# Create an empty list and then populate with dictionaries

In [10]:
books_df_original=pd.DataFrame(books)
ids=list(range(0, int(books_df_original.shape[0])))
books_df_original["book_id"]=ids
books_df_original.head(3)

,name,details_link,authors,avg_rating,count_ratings,editions_count,genre,reviews,price,price_type,pages_count,year_first_published,book_id
0,The Animals' Lawsuit Against Humanity: An Illu...,https://www.goodreads.com/book/show/183006.The...,"[Ikhwan al-Safa, Rabbi Dan Bridge, Rabbi Kalon...",4.25,171,13,10th-century,24,0.00,[Kindle],170,950,0
1,The Animals' Lawsuit Against Humanity: An Illu...,https://www.goodreads.com/book/show/19544232-t...,"[Ikhwan al-Safa, Umm Kulthum, Anson Laytner]",4.23,26,1,10th-century,3,None,None,170,2005,1
2,"The Formation of the Sunni Schools of Law, 9th...",https://www.goodreads.com/book/show/4320749-th...,[Christopher Melchert],4.00,11,4,10th-century,3,None,None,244,1997,2


In [11]:
columns=books_df_original.columns
columns

Index(['name', 'details_link', 'authors', 'avg_rating', 'count_ratings',
       'editions_count', 'genre', 'reviews', 'price', 'price_type',
       'pages_count', 'year_first_published', 'book_id'],
      dtype='object')

In [12]:
# find none and n/a
original_row_count=books_df_original.shape[0]
for i in columns:
    count_na=books_df_original[books_df_original[i].isna()==True].shape[0]
    print(f"{i}: {count_na} out of {original_row_count}")

name: 0 out of 400
details_link: 0 out of 400
authors: 0 out of 400
avg_rating: 0 out of 400
count_ratings: 0 out of 400
editions_count: 0 out of 400
genre: 0 out of 400
reviews: 300 out of 400
price: 369 out of 400
price_type: 369 out of 400
pages_count: 303 out of 400
year_first_published: 304 out of 400
book_id: 0 out of 400


In [13]:
books_df_original = books_df_original.drop_duplicates(subset=['name', 'genre'], keep='first')
titles_duplicates = books_df_original[books_df_original.duplicated(subset=['name', 'genre'], keep=False)]
print(titles_duplicates.shape[0])

0


In [14]:
books_df=books_df_original[['book_id','name', 'avg_rating', 'count_ratings',
       'genre', 'reviews', 'pages_count',
       'year_first_published']].copy()
books_df.rename(columns={'name': 'book_title'}, inplace=True)
books_df.head(3)

,book_id,book_title,avg_rating,count_ratings,genre,reviews,pages_count,year_first_published
0,0,The Animals' Lawsuit Against Humanity: An Illu...,4.25,171,10th-century,24,170,950
2,2,"The Formation of the Sunni Schools of Law, 9th...",4.00,11,10th-century,3,244,1997
3,3,The Fihrist: a 10th Century AD Survey of Islam...,4.00,8,10th-century,0,Hardcover,1970


In [15]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 390 entries, 0 to 399
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   book_id               390 non-null    int64  
 1   book_title            390 non-null    object 
 2   avg_rating            390 non-null    float64
 3   count_ratings         390 non-null    int64  
 4   genre                 390 non-null    object 
 5   reviews               96 non-null     object 
 6   pages_count           93 non-null     object 
 7   year_first_published  92 non-null     object 
dtypes: float64(1), int64(2), object(5)
memory usage: 27.4+ KB


In [16]:
# authors to split into separate list and add a linkage to books
authors_list=[]
book_auth_dict_list=[]
id_counter=1
for index, row in books_df_original.iterrows():
    for i in row["authors"]:     
#  change several spaces into single one
        name_single_space = i.strip()  
        while '  ' in name_single_space:
            name_single_space = name_single_space.replace('  ', ' ')
#  split several autors into several records
        split_authors = re.split(r',\s(?=[A-Z])', name_single_space)
        for j in range(len(split_authors)):
            authors_list.append(split_authors[j])
            # create book_author link
            book_auth_dict = {
                'book_id': row['book_id'],
                'author': split_authors[j],
                'auth_book_ID': id_counter
            }
            id_counter+=1
            book_auth_dict_list.append(book_auth_dict)
authors_dict_list=[]
for i in set(authors_list):
    authors_dict={}
    authors_dict['name']=i
    authors_dict_list.append(authors_dict)

authors_df=pd.DataFrame(authors_dict_list)
authors_df.head(3)

,name
0,Peter Adamson
1,Robert Muchembled
2,Irving Stone


In [17]:
book_auth_df = pd.DataFrame(book_auth_dict_list)
book_auth_df.head(3)

,book_id,author,auth_book_ID
0,0,Ikhwan al-Safa,1
1,0,Rabbi Dan Bridge,2
2,0,Rabbi Kalonymus,3


In [18]:
genres_dict_list=[]
id_counter=0
for i in range(len(genre_list)):
    genres_dict={
        'id':id_counter,
        'genre':genre_list[i],
        'genre_link':genre_links_list[i]
    }
    id_counter+=1
    genres_dict_list.append(genres_dict)

genres_df=pd.DataFrame(genres_dict_list)
genres_df.head(4)

,id,genre,genre_link
0,0,10th-century,https://www.goodreads.com/search?q=10th-century
1,1,11th-century,https://www.goodreads.com/search?q=11th-century
2,2,12th-century,https://www.goodreads.com/search?q=12th-century
3,3,13th-century,https://www.goodreads.com/search?q=13th-century


In [19]:
prices_dict_list=[]
# id_counter=0
for index, row in books_df_original.iterrows():
    prices_dict={
        'id':id_counter,
        'book_id':row['book_id'],
        'amount':row['price'],
        'type':row['price_type']
    }
    id_counter+=1
    prices_dict_list.append(prices_dict)

prices_df=pd.DataFrame(prices_dict_list)
prices_df.head(4)

,id,book_id,amount,type
0,1341,0,0.00,[Kindle]
1,1342,2,None,None
2,1343,3,None,None
3,1344,4,35.10,[]


In [20]:
genres_df.to_csv('ResourcesScraping/genres.csv', encoding='utf8', index=False)

In [21]:
# Write the data to a CSV 
books_df_original.to_csv('ResourcesScraping/books_original.csv', encoding='utf8', index=False)
books_df.to_csv('ResourcesScraping/books_table.csv', encoding='utf8', index=False)
authors_df.to_csv('ResourcesScraping/authors_table.csv', encoding='utf8', index=False)
book_auth_df.to_csv('ResourcesScraping/book_auth_table.csv', encoding='utf8', index=False)
prices_df.to_csv('ResourcesScraping/prices_table.csv', encoding='utf8', index=False)


In [22]:
browser.quit()